# 521153S Deep Learning Final Project

In [ ]:
# import necessary packages
import os
import requests
import zipfile
import sys
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
import torchvision
from PIL import Image
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import transforms, datasets
from torchvision.io import read_image
from torchvision.models import resnet18, resnet34, resnet50, resnet101, resnet152, ResNet18_Weights, ResNet34_Weights, ResNet50_Weights, ResNet101_Weights, ResNet152_Weights
import gdown
import urllib
import os
import tarfile

In [ ]:
Download val, test and traininig data
# download the dataset
val_url = 'https://drive.google.com/u/0/uc?id=1hSMUMj5IRpf-nQs1OwgiQLmGZCN0KDWl'
train_url = 'https://drive.google.com/u/0/uc?id=107FTosYIeBn5QbynR46YG91nHcJ70whs'
test_url = 'https://drive.google.com/u/0/uc?id=1yKyKgxcnGMIAnA_6Vr2ilbpHMc9COg-v'
eurosat_url = 'https://zenodo.org/records/7711810/files/EuroSAT_RGB.zip?download=1'

val_file = './data/val.tar'
train_file = './data/train.tar'
test_file = './data/test.tar'
eurosat_file = './data/eurosatrgb.zip'


if not os.path.exists('./data'):
    print('Creating data directory')
    os.mkdir('./data')

if not os.path.exists('./data/val.tar'):
    print('Downloading val.tar')
    gdown.download(val_url, val_file)
    val_tar = tarfile.open(val_file)
    val_tar.extractall('./data/')
    val_tar.close()

if not os.path.exists(train_file):
    print('Downloading train.tar')
    gdown.download(train_url, train_file)
    train_tar = tarfile.open(train_file)
    train_tar.extractall('./data/')
    train_tar.close()

if not os.path.exists(test_file):
    print('Downloading test.tar')
    gdown.download(test_url, test_file)
    test_tar = tarfile.open(test_file)
    test_tar.extractall('./data/')
    test_tar.close()

if not os.path.exists(eurosat_file):
    print('Downloading EuroSAT_RGB.zip')
    response = urllib.request.urlretrieve(eurosat_url, eurosat_file)
    eurosat_zip = zipfile.ZipFile(eurosat_file)
    eurosat_zip.extractall('./data/eurosat')
    eurosat_zip.close()

In [ ]:
# The following code will switch to cuda device automatically to accelerate your code if gpu is available in your environment.

# To enable GPU in Google Colab, you can navigate to `Edit → Notebook Settings → Hardware Accelerator` , and then select `T4 GPU` or `TPU`.
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# If you encounter some issues regarding cuda device, e.g., "RuntimeError: CUDA Out of memory error",
# try to switch the device to cpu by using the following code

# device = torch.device('cpu')
print('Device:', device)

In [ ]:
Pretrained model
# Image Size
image_size = 32

# Batch size during training
batch_size = 100
num_workers = 1

# Learning rate for optimizers
lr = 0.0002

# Number of training epochs
num_epochs = 30
model_resnet18 = torch.hub.load('pytorch/vision:v0.10.0', 'resnet18', pretrained=True)
model_resnet34 = torch.hub.load('pytorch/vision:v0.10.0', 'resnet34', pretrained=True)
model_resnet50 = torch.hub.load('pytorch/vision:v0.10.0', 'resnet50', pretrained=True)
model_resnet101 = torch.hub.load('pytorch/vision:v0.10.0', 'resnet101', pretrained=True)
model_resnet152 = torch.hub.load('pytorch/vision:v0.10.0', 'resnet152', pretrained=True)

train_dataset = torchvision.datasets.ImageFolder(
    root='./data/train',
    transform=transforms.Compose([
        transforms.Resize(image_size),
        transforms.CenterCrop(image_size),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
    ])
)

test_dataset = torchvision.datasets.ImageFolder(
    root='./data/test',
    transform=transforms.Compose([
        transforms.Resize(image_size),
        transforms.CenterCrop(image_size),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
    ])
)

validation_dataset = torchvision.datasets.ImageFolder(
    root='./data/val',
    transform=transforms.Compose([
        transforms.Resize(image_size),
        transforms.CenterCrop(image_size),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
    ])
)


dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers)
dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers)
dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers)

In [ ]:
img = read_image("test/assets/encode_jpeg/grace_hopper_517x606.jpg")

# Step 1: Initialize model with the best available weights
weights = ResNet50_Weights.DEFAULT
model = resnet50(weights=weights)
model.eval()

# Step 2: Initialize the inference transforms
preprocess = weights.transforms()

# Step 3: Apply inference preprocessing transforms
batch = preprocess(img).unsqueeze(0)

# Step 4: Use the model and print the predicted category
prediction = model(batch).squeeze(0).softmax(0)
class_id = prediction.argmax().item()
score = prediction[class_id].item()
category_name = weights.meta["categories"][class_id]
print(f"{category_name}: {100 * score:.1f}%")

In [ ]:
# Define transformations for data augmentation and normalization
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images to fit ResNet input size
    transforms.ToTensor(),          # Convert images to PyTorch tensors
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))  # Normalize images
])

# Load miniImageNet train, validation, and test datasets
train_dataset = datasets.ImageFolder('path_to_miniImageNet_train', transform=transform)
val_dataset = datasets.ImageFolder('path_to_miniImageNet_val', transform=transform)
test_dataset = datasets.ImageFolder('path_to_miniImageNet_test', transform=transform)

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64)
test_loader = DataLoader(test_dataset, batch_size=64)

# Define the model (ResNet18 in this example)
model = torchvision.models.resnet18(pretrained=True)
# Modify the last layer to match the number of classes in miniImageNet
model.fc = nn.Linear(model.fc.in_features, len(train_dataset.classes))

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# Training loop
num_epochs = 10
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

for epoch in range(num_epochs):
    model.train()
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

# Evaluation on validation set
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in val_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

val_accuracy = correct / total
print(f"Validation Accuracy: {val_accuracy * 100:.2f}%")

# Evaluation on test set
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

test_accuracy = correct / total
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

In [ ]:
# Define transformations for data augmentation and normalization
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images to fit ResNet input size
    transforms.ToTensor(),          # Convert images to PyTorch tensors
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))  # Normalize images
])

# Load miniImageNet train, validation, and test datasets
train_dataset = datasets.ImageFolder('path_to_miniImageNet_train', transform=transform)
val_dataset = datasets.ImageFolder('path_to_miniImageNet_val', transform=transform)
test_dataset = datasets.ImageFolder('path_to_miniImageNet_test', transform=transform)

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64)
test_loader = DataLoader(test_dataset, batch_size=64)

# Define the model (ResNet18 in this example)
model = torchvision.models.resnet18(pretrained=True)
# Modify the last layer to match the number of classes in miniImageNet
model.fc = nn.Linear(model.fc.in_features, len(train_dataset.classes))

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# Training loop
num_epochs = 10
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

for epoch in range(num_epochs):
    model.train()
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

# Evaluation on validation set
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in val_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

val_accuracy = correct / total
print(f"Validation Accuracy: {val_accuracy * 100:.2f}%")

# Evaluation on test set
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

test_accuracy = correct / total
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

